In [2]:
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import torch.optim as optim
import torch.nn.functional as F
from imp import reload
import nice_layers as vnn
reload(vnn)

<module 'nice_layers' from '/home/davidclark/Projects/VectorizedNets/nice_layers.py'>

In [88]:
def convolutional_outer_product(x1, x2, kernel_size):
    #assumes stride, padding, dilation are all defaults!
    #x1 = (batch_size, in_channels, h_in, w_in)
    #x2 = (batch_size, out_channels, h_out, w_out)
    #out = (batch_size, out_chanels, in_channels, kernel_size, kernel_size)
    
    batch_size = x1.shape[0]
    if x2.shape[0] != batch_size:
        raise ValueError("x1 and x2 must have the same batch size")
    in_channels = x1.shape[1]
    out_channels = x2.shape[1]
    h_in, w_in = x1.shape[2:]
    h_out, w_out = x2.shape[2:]
    
    w_out_expected = w_in - kernel_size + 1
    h_out_expected = h_in - kernel_size + 1
    if (h_out_expected != h_out) or (w_out_expected != w_out):
        raise ValueError("invalid kernel size")

    x1_prime = x1.permute(1, 0, 2, 3)
    x2_prime = x2.view(batch_size*out_channels, h_out, w_out).unsqueeze(1)
    out_prime = F.conv2d(input=x1_prime, weight=x2_prime, groups=batch_size)
    out = out_prime.view(in_channels, batch_size, out_channels, kernel_size, kernel_size).permute(1, 2, 0, 3, 4)
    return out




In [93]:
x1 = torch.randn(256, 5, 28, 28)
x2 = torch.randn(256, 3, 22, 22)

out = convolutional_outer_product(x1, x2, 7)
out.shape

torch.Size([256, 3, 5, 7, 7])

In [83]:
for i in range(out_channels):
    for j in range(in_channels):
        res = out[-1, i, j]
        target = 2*(i+1)*(j+1)*22*22
        print(res[0, 0], target)


tensor(968.) 968
tensor(1936.) 1936
tensor(2904.) 2904
tensor(3872.) 3872
tensor(1936.) 1936
tensor(3872.) 3872
tensor(5808.) 5808
tensor(7744.) 7744
tensor(2904.) 2904
tensor(5808.) 5808
tensor(8712.) 8712
tensor(11616.) 11616
tensor(3872.) 3872
tensor(7744.) 7744
tensor(11616.) 11616
tensor(15488.) 15488
tensor(4840.) 4840
tensor(9680.) 9680
tensor(14520.) 14520
tensor(19360.) 19360


In [7]:
input = torch.randn(256, 10, 150)
output = model(input)
output.shape

torch.Size([256, 10, 50])

In [8]:
model

Sequential(
  (0): VecLinear()
  (1): VecLinear()
)

In [191]:
reload(vnn)

model = nn.Sequential(vnn.Conv2d(10, 30, 6, 5),
                      vnn.tReLU(),
                      vnn.AvgPool2d(2),
                      vnn.Conv2d(10, 6, 16, 5),
                      vnn.tReLU(),
                      vnn.AvgPool2d(2),
                      vnn.Conv2d(10, 16, 120, 5),
                      vnn.tReLU(),
                      vnn.Flatten(),
                      vnn.Linear(10, 120, 84),
                      vnn.tReLU(),
                      vnn.Dropout(),
                      vnn.Linear(10, 84, 1))

In [192]:
input = torch.randn(1, 10, 30, 32, 32)


In [193]:
output = model(input)
output.shape

Instantiated t with shape (10, 6, 28, 28)
Instantiated t with shape (10, 16, 10, 10)
Instantiated t with shape (10, 120, 1, 1)
Instantiated t with shape (10, 84)


torch.Size([1, 10, 1])

In [187]:
for module in model: module.post_step_callback()


In [120]:
do = nn.Dropout(p=0.5)
x = torch.ones(5, 3)
do(x)

tensor([[2., 0., 0.],
        [2., 2., 2.],
        [0., 0., 0.],
        [0., 0., 0.],
        [2., 0., 2.]])

In [166]:
class Dropout(nn.Module):
    def __init__(self, p=0.5):
        super().__init__()
        self.p = p
    def forward(self, input):
        if self.training:
            mask_shape = (input.shape[0],) + input.shape[2:]
            mask = (torch.rand(mask_shape, device=input.device) > self.p).float()
            output = input * mask[:, None] * (1. / (1. - self.p))
            return output
        else:
            return input

In [175]:
do = Dropout(p=0.9)
x = torch.ones(3, 10, 5)
do.train()
do(x)

tensor([[[ 0., 10.,  0.,  0.,  0.],
         [ 0., 10.,  0.,  0.,  0.],
         [ 0., 10.,  0.,  0.,  0.],
         [ 0., 10.,  0.,  0.,  0.],
         [ 0., 10.,  0.,  0.,  0.],
         [ 0., 10.,  0.,  0.,  0.],
         [ 0., 10.,  0.,  0.,  0.],
         [ 0., 10.,  0.,  0.,  0.],
         [ 0., 10.,  0.,  0.,  0.],
         [ 0., 10.,  0.,  0.,  0.]],

        [[ 0.,  0.,  0.,  0., 10.],
         [ 0.,  0.,  0.,  0., 10.],
         [ 0.,  0.,  0.,  0., 10.],
         [ 0.,  0.,  0.,  0., 10.],
         [ 0.,  0.,  0.,  0., 10.],
         [ 0.,  0.,  0.,  0., 10.],
         [ 0.,  0.,  0.,  0., 10.],
         [ 0.,  0.,  0.,  0., 10.],
         [ 0.,  0.,  0.,  0., 10.],
         [ 0.,  0.,  0.,  0., 10.]],

        [[ 0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,

In [168]:
do.

tensor([[[0., 2., 2., 2., 2.],
         [0., 2., 2., 2., 2.],
         [0., 2., 2., 2., 2.],
         [0., 2., 2., 2., 2.],
         [0., 2., 2., 2., 2.],
         [0., 2., 2., 2., 2.],
         [0., 2., 2., 2., 2.],
         [0., 2., 2., 2., 2.],
         [0., 2., 2., 2., 2.],
         [0., 2., 2., 2., 2.]],

        [[0., 2., 0., 0., 0.],
         [0., 2., 0., 0., 0.],
         [0., 2., 0., 0., 0.],
         [0., 2., 0., 0., 0.],
         [0., 2., 0., 0., 0.],
         [0., 2., 0., 0., 0.],
         [0., 2., 0., 0., 0.],
         [0., 2., 0., 0., 0.],
         [0., 2., 0., 0., 0.],
         [0., 2., 0., 0., 0.]],

        [[0., 0., 2., 2., 2.],
         [0., 0., 2., 2., 2.],
         [0., 0., 2., 2., 2.],
         [0., 0., 2., 2., 2.],
         [0., 0., 2., 2., 2.],
         [0., 0., 2., 2., 2.],
         [0., 0., 2., 2., 2.],
         [0., 0., 2., 2., 2.],
         [0., 0., 2., 2., 2.],
         [0., 0., 2., 2., 2.]]])